# AmpForm demo [BESIII Collaboration Winter Meeting 2021](https://indico.ihep.ac.cn/event/15291)

This notebook accompanies [these slides](https://docs.google.com/presentation/d/e/2PACX-1vTMb3vsOqQUI_A3LYMs0iBvFwuIzyf49rG-PDXpR2TzwXJ4hkg-NzPB_Mslv7DcZuV1Tzm7duZEtI8q/pub). They were presented during the BESIII Collaboration Meeting on 29 November 2021.

Related notebooks for this presentation:
- [QRules demo](./qrules.ipynb)
- [TensorWaves demo](./tensorwaves.ipynb)

For more extensive examples, see **[ampform.rtfd.io](https://ampform.readthedocs.io)**.

## Install dependencies

In [ ]:
%pip install -q ampform[viz]==0.11.*

In [ ]:
%load_ext autoreload
%autoreload
import ampform
import graphviz
import qrules
import symplot
import sympy as sp
from ampform.dynamics import BlattWeisskopfSquared, CoupledWidth
from ampform.dynamics.builder import (
    create_analytic_breit_wigner,
    create_relativistic_breit_wigner_with_ff,
)
from ampform.dynamics.kmatrix import NonRelativisticKMatrix, RelativisticKMatrix
from IPython.display import Math, display
from sympy.physics.quantum.spin import WignerD

## $K$-matrix expressions

In [ ]:
n = sp.Symbol("n_R")
matrix = RelativisticKMatrix.formulate(
    n_channels=1,
    n_poles=n,
)
matrix[0, 0]

In [ ]:
matrix = NonRelativisticKMatrix.formulate(
    n_channels=2,
    n_poles=1,
).doit()
matrix[0, 0].simplify()

## Example $D^0 \to K^0 K^- K^+$

### Generate transitions

In [ ]:
reaction = qrules.generate_transitions(
    initial_state="D0",
    final_state=["K0", "K-", "K+"],
    allowed_intermediate_particles=["a(0)(980)0", "a(0)(1450)0", "phi(1020)"],
    formalism="helicity",
)

In [ ]:
dot = qrules.io.asdot(reaction, collapse_graphs=True, render_final_state_id=True)
graphviz.Source(dot)

Note that one of the resonances, $a_0(980)$, lies **below threshold**, which means we should parametrize it with an analytic continuation.

In [ ]:
PDG = qrules.load_pdg()
PDG["a(0)(980)0"].mass < PDG["K-"].mass + PDG["K-"].mass

In [ ]:
builder = ampform.get_builder(reaction)
resonances = reaction.get_intermediate_particles()
for p in resonances:
    builder.set_dynamics(p.name, create_relativistic_breit_wigner_with_ff)
builder.set_dynamics("a(0)(980)0", create_analytic_breit_wigner)
model = builder.formulate()

In [ ]:
amplitude_expressions = {
    expr: sp.Symbol(name)
    for name, expr in model.components.items()
    if name.startswith("A")
}
top_expr = model.expression.xreplace(amplitude_expressions)
top_expr

### Examine one of the amplitudes

In [ ]:
some_amplitude = model.components[
    R"A_{D^{0}_{0} \to K^{0}_{0} \phi(1020)_{0}; \phi(1020)_{0} \to K^{+}_{0} K^{-}_{0}}"
]

In [ ]:
def round_nested(expression, n_decimals):
    for node in sp.preorder_traversal(expression):
        if node.free_symbols:
            continue
        if isinstance(node, (float, sp.Float)):
            expression = expression.xreplace({node: node.n(n_decimals)})
    return expression


A_step1 = some_amplitude
A_step2 = symplot.partial_doit(A_step1, (BlattWeisskopfSquared, WignerD))
A_step2 = symplot.partial_doit(A_step2, CoupledWidth)
substitutions = {
    sp.Symbol("m_0", real=True): PDG["K0"].mass,
    sp.Symbol("m_1", real=True): PDG["K-"].mass,
    sp.Symbol("m_2", real=True): PDG["K+"].mass,
    **model.parameter_defaults,
    sp.sqrt(2): sp.sqrt(2).n(),
}
A_step3 = some_amplitude.doit().xreplace(substitutions)
A_step3 = round_nested(A_step3, n_decimals=2)
A_step3 = sp.posify(A_step3)[0]
display(
    A_step1,
    Math("=" + sp.latex(A_step2)),
    Math("=" + sp.latex(A_step3)),
)

### Visualize expression tree

In [ ]:
dot = sp.dotprint(A_step3)
graphviz.Source(dot)